In [ ]:
# default_exp core.train_detectron2 

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Train Detectron2
> Train a new model on our new keypoint dataset in COCO keypoint detection format 

In [ ]:
#export
# Import some common libraries
import numpy as np
import matplotlib.pyplot as plt
# import os, json, cv2, random

# Import some common detectron2 utilities
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

## Load data
> This requires that your data is in COCO format.

In [ ]:
#export
# register_coco_instances("my_dataset_train", {}, "json_annotation_train.json", "path/to/image/dir")
# register_coco_instances("my_dataset_val", {}, "json_annotation_val.json", "path/to/image/dir")

In [ ]:
#export
# TODO: Add names from dataframe. 
# TODO: Update to match the new model
keypoint_names = [
    'RWaist', # 1
    'LWaist', # 2
    'RKnee',  # 3
    'LKnee',  # 4
    'RWrist', # 5
    'LWrist', # 6
    'RElbow', # 7
    'LElbow', # 8
    'RForefoot2', # 9
    'RAnkleOut', # 10
    'LForefoot2', # 11
    'LAnkleOut', # 12 
    'WaistBack', # 13
    'RHand2', # 14
    'RShoulderTop', # 15
    'LHand2', # 16
    'LShoulderTop', # 17
    'SpineThoracic12', # 18
    'SpineThoracic2', # 19
    'HeadFront' # 20
]
keypoint_flip_map = [
    ('RShoulderTop', 'LShoulderTop'), ('RElbow', 'LElbow'), ('RWrist', 'LWrist'),
    ('RHand2', 'LHand2'), ('RWaist', 'LWaist'), ('RKnee', 'LKnee'), 
    ('RAnkleOut', 'LAnkleOut'), ('RForefoot2', 'LForefoot2')
]
# MetadataCatalog.get("my_dataset_train").keypoint_names = keypoint_names
# MetadataCatalog.get("my_dataset_train").keypoint_flip_map = keypoint_flip_map
# MetadataCatalog.get("my_dataset_train").evaluator_type = "coco"

# dataset_dicts = DatasetCatalog.get("my_dataset_train")

In [ ]:
#hide
# To verify the data loading is correct, 
# let's visualize the annotations of randomly selected samples in the dataset:

# metadata = MetadataCatalog.get("my_dataset_train")

# def cv2_imshow(im):
#     im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
#     plt.figure(), plt.imshow(im), plt.axis('off');

# for d in random.sample(dataset_dicts, 5):
#     img = cv2.imread(d["file_name"])
#     visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.5)   
#     vis = visualizer.draw_dataset_dict(d)
#     cv2_imshow(vis.get_image()[:, :, ::-1])

## Train

In [ ]:
#export 
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
# cfg.DATASETS.TRAIN = ("my_dataset_train",)
# cfg.DATASETS.TEST = ("my_dataset_test")
cfg.DATALOADER.NUM_WORKERS = 2
# Let training initialize from model zoo
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml") 
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # Pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations
cfg.SOLVER.STEPS = []        # Do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128 # NOTE: Increase if possible  
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Only have one class
cfg.MODEL.RETINANET.NUM_CLASSES = 1
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 20
# More info on below see section 1.3: https://cocodataset.org/#keypoints-eval
cfg.TEST.KEYPOINT_OKS_SIGMAS = np.ones((20, 1), dtype=float).tolist()

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
# #export
# trainer = DefaultTrainer(cfg) 
# trainer.resume_or_load(resume=False)
# trainer.train()

In [ ]:
# #hide
# # Look at training curves in tensorboard:
# %load_ext tensorboard
# %tensorboard --logdir output